In [75]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
import sys
sys.path.insert(0, '..//src/')
import pandas as pd
from project_settings import settings
from pathlib import Path
import plotly.graph_objects as go
# import kaleido
# print(kaleido.__version__)

In [77]:
elec_hh_data = pd.read_csv(Path(r'../data/consumption.csv'), index_col=0)
elec_hh_data['interval_start'] = pd.to_datetime(elec_hh_data['interval_start'], format="%Y-%m-%dT%H:%M:%S%z", utc=True)
elec_hh_data['interval_end'] = pd.to_datetime(elec_hh_data['interval_end'], format="%Y-%m-%dT%H:%M:%S%z", utc=True)
elec_hh_data.set_index('interval_start', inplace=True)
elec_hh_data = elec_hh_data[['consumption']]
elec_hh_data.columns = ['Electricity consumption (kWh)']

In [90]:
def create_overall_chart(dataf:pd.DataFrame):
    # Resample data
    daily_consumption = dataf['Electricity consumption (kWh)'].resample('d').mean()
    weekly_consumption = dataf['Electricity consumption (kWh)'].resample('W').mean()
    monthly_consumption = dataf['Electricity consumption (kWh)'].resample('ME').mean()

    # Create traces
    trace_daily = go.Scatter(x=daily_consumption.index, y=daily_consumption, mode='lines', name='Daily Consumption')
    trace_weekly = go.Scatter(x=weekly_consumption.index, y=weekly_consumption, mode='lines', name='Weekly Consumption')
    trace_monthly = go.Scatter(x=monthly_consumption.index, y=monthly_consumption, mode='lines', name='Monthly Consumption')

    # Create figure
    fig = go.Figure()

    # Add traces to figure
    fig.add_trace(trace_daily)
    fig.add_trace(trace_weekly)
    fig.add_trace(trace_monthly)

    # Update layout
    fig.update_layout(
        title='Electricity Consumption Over Time',
        xaxis_title='Date',
        yaxis_title='Electricity consumption (kWh)',
        legend_title='Resample Frequency',
        template='plotly'
    )
    return fig


# Show plot
fig = create_overall_chart(elec_hh_data)
fig.show()

In [79]:
## Peak electricity import
elec_hh_data['Electricity consumption (kWh)'].max()

7.385

In [ ]:
## Decreasing peak

In [80]:
after_battery_data = elec_hh_data[['Electricity consumption (kWh)']].copy()
after_battery_data = after_battery_data.rolling(window=24, center=True, min_periods=0).mean()
daily_avg_hh_consumption_after_improvements = after_battery_data.groupby([elec_hh_data.index.hour]).agg({'Electricity consumption (kWh)': 'mean'})
daily_avg_hh_consumption_before_improvements = elec_hh_data.groupby([elec_hh_data.index.hour]).agg({'Electricity consumption (kWh)': 'mean'})

In [81]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=daily_avg_hh_consumption_before_improvements.index, y=daily_avg_hh_consumption_before_improvements['Electricity consumption (kWh)'], mode='lines', name='Before Improvements'))
fig.add_trace(go.Scatter(x=daily_avg_hh_consumption_after_improvements.index, y=daily_avg_hh_consumption_after_improvements['Electricity consumption (kWh)'], mode='lines', name='After Improvements'))

fig.update_layout(
    title='Average Hourly Consumption Before and After Improvements',
    xaxis_title='Hour of Day',
    yaxis_title='Consumption (kWh)',
    template='plotly'
)

In [91]:
carbon_intensity = pd.read_csv(settings.PATH_SAVE_DATA/'carbon_intensity.csv', index_col=0)
carbon_intensity.index = pd.to_datetime(carbon_intensity.index, utc=True)
carbon_intensity.dropna(inplace=True)

In [92]:
carbon_emission_profile = pd.merge(elec_hh_data, carbon_intensity, left_index=True, right_index=True, how='left')
carbon_emission_profile['Total carbon emissions (gCO2)'] = carbon_emission_profile['Electricity consumption (kWh)']*carbon_emission_profile['carbon_intensity (gCO2/kWh)']


In [99]:
## Annual tCO2 emissions
carbon_emission_profile['Total carbon emissions (gCO2)'].resample('YE').sum()/1000000

interval_start
2023-12-31 00:00:00+00:00    1.691159
2024-12-31 00:00:00+00:00    0.717511
Freq: YE-DEC, Name: Total carbon emissions (gCO2), dtype: float64

In [93]:
carbon_emission_profile['Total carbon emissions (gCO2)'].plot()

In [115]:
avg_day = carbon_emission_profile.groupby([carbon_emission_profile.index.weekday, carbon_emission_profile.index.hour]).mean()['Total carbon emissions (gCO2)'].to_frame().unstack(level=0)
avg_day.columns = avg_day.columns.droplevel(0)


In [117]:
fig = go.Figure()

for c in avg_day.columns:    
    fig.add_trace(go.Scatter(x=avg_day.index, y=avg_day[c], mode='lines', name=f'Day {c}'))

fig.update_layout(
    title=f'Average Carbon Emissions by Day of Week',
    xaxis_title='Hour of Day',
    yaxis_title='Carbon Emissions (gCO2)',
    template='plotly'
)
fig.show()